In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import numpy as np


In [5]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [6]:
X_train.shape

(60000, 28, 28)

In [7]:
num_pixels = X_train.shape[1] * X_train.shape[2]
n_channels = 1
def preprocess(matrix):
    return matrix.reshape(matrix.shape[0], \
                          n_channels, \
                          matrix.shape[1], \
                          matrix.shape[2]
                         ).astype('float32') / 255.

In [8]:
X_train, X_test = preprocess(X_train), preprocess(X_test)

In [9]:
X_train.shape, X_train.dtype, np.max(X_train)

((60000, 1, 28, 28), dtype('float32'), 1.0)

In [10]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_train.shape[1]

In [11]:
y_train.shape

(60000, 10)

In [12]:
def baseline_model():
    # Tworzenie modelu
    model = Sequential()
    model.add(Flatten(input_shape=(1, 28, 28)))
    model.add(Dense(num_pixels, init='normal', activation='relu'))
    model.add(Dense(num_classes, init='normal', activation='softmax'))
    # Kompilacja modelu
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
def convolution_small():
    # Tworzenie modelu
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Kompilacja modelu
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [14]:
def convolution_large():
    # Tworzenie modelu
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Kompilacja modelu
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
# Budowanie modelu
np.random.seed(101)
models = [('baseline', baseline_model()), 
          ('small', convolution_small()), 
          ('large', convolution_large())]

for name, model in models:
    print("Używany model:", name)
    # Dopasowywanie modelu
    model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=100, verbose=2)
    # Ostateczna ocena modelu
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Błąd bazowy: %.2f%%" % (100-scores[1]*100))
    print()

ValueError: Negative dimension size caused by subtracting 5 from 1 for 'Conv2D_1' (op: 'Conv2D') with input shapes: [?,1,28,28], [5,5,28,32].